<a href="https://colab.research.google.com/github/vpcano/CAP_P2/blob/main/Ej2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
!mkdir -p src
!rm -rf src/*
!cd src && wget --no-check-certificate https://raw.githubusercontent.com/nothings/stb/master/stb_image.h
!cd src && wget --no-check-certificate https://raw.githubusercontent.com/nothings/stb/master/stb_image_write.h

--2023-10-17 10:13:02--  https://raw.githubusercontent.com/nothings/stb/master/stb_image.h
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 284733 (278K) [text/plain]
Saving to: ‘stb_image.h’

stb_image.h         100%[===================>] 278.06K  --.-KB/s    in 0.004s  

2023-10-17 10:13:02 (63.5 MB/s) - ‘stb_image.h’ saved [284733/284733]

--2023-10-17 10:13:03--  https://raw.githubusercontent.com/nothings/stb/master/stb_image_write.h
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71221 (70K) [text/plain]
Saving to: ‘stb_image_

## CPU Version (with intrinsics)

In [39]:
%%writefile src/greyScale_cpu.c
    #include <stdio.h>
    #include <stdint.h>
    #include <math.h>
    #include <sys/time.h>
    #include <x86intrin.h>
    #define STB_IMAGE_IMPLEMENTATION
    #include "stb_image.h"
    #define STB_IMAGE_WRITE_IMPLEMENTATION
    #include "stb_image_write.h"



    int main(int nargs, char **argv)
    {
        int width, height, nchannels;
        struct timeval fin,ini;
        const __m256 coefficients = {0.2989, 0.5870, 0.1140, 0.0, 0.2989, 0.5870, 0.1140, 0.0};

        if (nargs < 2)
        {
            printf("Usage: %s <image1> [<image2> ...]\n", argv[0]);
        }
        // For each image
        // Bucle 0
        for (int file_i = 1; file_i < nargs; file_i++)
        {
            printf("[info] Processing %s\n", argv[file_i]);
            /****** Reading file ******/
            uint8_t *rgb_image = stbi_load(argv[file_i], &width, &height, &nchannels, 4);
            if (!rgb_image)
            {
                perror("Image could not be opened");
            }

            /****** Allocating memory ******/
            // - RGB2Grey
            uint8_t *grey_image = malloc(width * height);
            if (!grey_image)
            {
                perror("Could not allocate memory");
            }

            // - Filenames
            for (int i = strlen(argv[file_i]) - 1; i >= 0; i--)
            {
                if (argv[file_i][i] == '.')
                {
                    argv[file_i][i] = 0;
                    break;
                }
            }

            char *grey_image_filename = 0;
            asprintf(&grey_image_filename, "%s_grey.jpg", argv[file_i]);
            if (!grey_image_filename)
            {
                perror("Could not allocate memory");
                exit(-1);
            }

            /****** Computations ******/
            printf("[info] %s: width=%d, height=%d, nchannels=%d\n", argv[file_i], width, height, nchannels);

            if (nchannels != 3 && nchannels != 4)
            {
                printf("[error] Num of channels=%d not supported. Only three (RGB), four (RGBA) are supported.\n", nchannels);
                continue;
            }

            gettimeofday(&ini,NULL);
            // RGB to grey scale
            for (int j = 0; j < height; j++)
            {
                for (int i = 0; i < width; i+=4)
                {
                    // Obtain first two pixels (64 bits) and store them as four 8-bit packed
                    // integers each in the lowest half of a 128 bit register
                    __m128i lo_two_pixels = _mm_loadl_epi64((__m128i*)(rgb_image + (i + j*width) * 4));
                    // Do the same with the next two pixels
                    __m128i hi_two_pixels = _mm_loadl_epi64((__m128i*)(rgb_image + (i + 2 + j*width) * 4));

                    // Extend each 8-bit packed integer from each pixel to a 32-bit packed
                    // integer and store each pair of pixels in a 256 bit register
                    __m256i extended_lo_two_pixels = _mm256_cvtepu8_epi32(lo_two_pixels);
                    __m256i extended_hi_two_pixels = _mm256_cvtepu8_epi32(hi_two_pixels);

                    // Convert each 32-bit packed integer to a 32-bit packed float
                    __m256 float_lo_two_pixels = _mm256_cvtepi32_ps(extended_lo_two_pixels);
                    __m256 float_hi_two_pixels = _mm256_cvtepi32_ps(extended_hi_two_pixels);

                    // Multiply each 32-bit packed float with the coefficients (defined
                    // at the begining)
                    __m256 prod_lo_two_pixels = _mm256_mul_ps(float_lo_two_pixels, coefficients);
                    __m256 prod_hi_two_pixels = _mm256_mul_ps(float_hi_two_pixels, coefficients);

                    // Horizontally add the registers in order to achieve the sum of the
                    // products for each pixel
                    __m256 sum = _mm256_hadd_ps(prod_lo_two_pixels, prod_hi_two_pixels);
                    sum = _mm256_hadd_ps(sum, sum);

                    // Reorder the sum results to keep the original pixel order
                    __m256 reordered_sum = _mm256_permutevar8x32_ps(sum, _mm256_set_epi32(7, 6, 3, 2, 5, 1, 4, 0));

                    // Take the four first 32-bit packed floats and convert them to
                    // 32-bit packed integers
                    __m128i grey_pixels = _mm_cvtps_epi32(_mm256_extractf128_ps(reordered_sum, 0));

                    // The following code is valid but requires too much instructions
                    // grey_image[i + j*width] = (uint8_t) _mm_extract_epi8(grey_pixels, 0);
                    // grey_image[i + j*width + 1] = (uint8_t) _mm_extract_epi8(grey_pixels, 4);
                    // grey_image[i + j*width + 2] = (uint8_t) _mm_extract_epi8(grey_pixels, 8);
                    // grey_image[i + j*width + 3] = (uint8_t) _mm_extract_epi8(grey_pixels, 12);

                    // This aproach results in less instructions: each 32-bit integer
                    // is packed into a 16-bit integer and stored next to each other
                    // at the begining of the register. Then the same is done but from
                    // a 16-bit integer to a 8-bit integer, so the result has the four
                    // pixels stored on the first 32 bit of the register. Finally, only
                    // one assignment needs to be done. The grey_image array is casted
                    // to a uint32_t pointer and its value at the corresponding index
                    // is assigned to the first four pixels
                    grey_pixels = _mm_packus_epi32(grey_pixels, grey_pixels);
                    grey_pixels = _mm_packus_epi16(grey_pixels, grey_pixels);
                    *((uint32_t*)(grey_image + i + j*width)) = _mm_extract_epi32(grey_pixels, 0);
                }
            }

            stbi_write_jpg(grey_image_filename, width, height, 1, grey_image, 10);
            free(rgb_image);

            gettimeofday(&fin,NULL);

            printf("Tiempo: %f\n", ((fin.tv_sec*1000000+fin.tv_usec)-(ini.tv_sec*1000000+ini.tv_usec))*1.0/1000000.0);
            free(grey_image_filename);
        }
    }

Writing src/greyScale_cpu.c


In [40]:
!gcc -O3 -march=native -fwhole-program -Wall -D_GNU_SOURCE -std=gnu99 -o src/greyScale_cpu src/greyScale_cpu.c -lm
!./src/greyScale_cpu images/FHD.jpg


src/greyScale_cpu.c: In function ‘main’:
src/greyScale_cpu.c:54:13: warning: ignoring return value of ‘asprintf’ declared with attribute ‘warn_unused_result’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-result-Wunused-result]8;;]
   54 |             asprintf(&grey_image_filename, "%s_grey.jpg", argv[file_i]);
      |             ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[info] Processing images/FHD.jpg
[info] images/FHD: width=1920, height=1080, nchannels=3
Tiempo: 0.040569


## GPU Version

In [57]:
%%writefile src/greyScale_gpu.cu
    #include <iostream>
    #include <stdint.h>
    #include <math.h>
    #include <sys/time.h>
    #define STB_IMAGE_IMPLEMENTATION
    #include "stb_image.h"
    #define STB_IMAGE_WRITE_IMPLEMENTATION
    #include "stb_image_write.h"
    #include "cuda.h"
    #include "cuda_runtime.h"

    #define BLOCK_SIZE_SIDE 16

    __global__ void rgbToGrayscale(uint8_t *rgb_image, uint8_t *grey_image, int width, int height, int nchannels) {
        int x = blockIdx.x * blockDim.x + threadIdx.x;
        int y = blockIdx.y * blockDim.y + threadIdx.y;

        if (x < width && y < height) {
            int grey_idx = y * width + x;
            int rgb_idx = grey_idx * nchannels;
            unsigned char *offset = rgb_image + rgb_idx;

            grey_image[grey_idx] = (int)(0.2989*offset[0] + 0.5870*offset[1] + 0.1140*offset[2]);
        }
    }

    int main(int argc, char *argv[]) {
        int width, height, nchannels;
        struct timeval fin,ini;

        if (argc < 2) {
            printf("Usage: %s <image1> [<image2> ...]\n", argv[0]);
            return -1;
        }

        // For each image
        for (int file_i = 1; file_i < argc; file_i++) {
            printf("[info] Processing %s\n", argv[file_i]);

            // Reading file
            uint8_t *rgb_image = stbi_load(argv[file_i], &width, &height, &nchannels, 4);
            if (!rgb_image) {
                perror("Image could not be opened");
            }

            // Allocating memory
            uint8_t *grey_image = (uint8_t*) malloc(width * height);
            if (!grey_image) {
                perror("Could not allocate memory");
                stbi_image_free(rgb_image);
            }

            // - Filenames
            for (int i = strlen(argv[file_i]) - 1; i >= 0; i--)
            {
                if (argv[file_i][i] == '.')
                {
                    argv[file_i][i] = 0;
                    break;
                }
            }

            char *grey_image_filename = 0;
            asprintf(&grey_image_filename, "%s_grey.jpg", argv[file_i]);
            if (!grey_image_filename) {
                perror("Could not allocate memory");
                stbi_image_free(rgb_image);
                exit(-1);
            }

            /****** Computations ******/
            printf("[info] %s: width=%d, height=%d, nchannels=%d\n", argv[file_i], width, height, nchannels);

            if (nchannels != 3 && nchannels != 4)
            {
                printf("[error] Num of channels=%d not supported. Only three (RGB), four (RGBA) are supported.\n", nchannels);
                continue;
            }

            // Allocate memory on GPU
            uint8_t *d_rgb_image;
            uint8_t *d_grey_image;
            cudaMalloc((void**)&d_rgb_image, width * height * 4);
            cudaMalloc((void**)&d_grey_image, width * height);

            gettimeofday(&ini,NULL);

            // Copy RGB image to GPU
            cudaMemcpy(d_rgb_image, rgb_image, width * height * 4, cudaMemcpyHostToDevice);

            // Calculate block and grid dimensions
            dim3 blockSize(BLOCK_SIZE_SIDE, BLOCK_SIZE_SIDE);
            dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);

            // Launch the kernel
            rgbToGrayscale<<<gridSize, blockSize>>>(d_rgb_image, d_grey_image, width, height, 4);

            // Copy the result back to the host
            cudaMemcpy(grey_image, d_grey_image, width * height, cudaMemcpyDeviceToHost);

            // Save the grayscale image
            stbi_write_jpg(grey_image_filename, width, height, 1, grey_image, 10);

            gettimeofday(&fin,NULL);
            printf("Tiempo: %f\n", ((fin.tv_sec*1000000+fin.tv_usec)-(ini.tv_sec*1000000+ini.tv_usec))*1.0/1000000.0);

            // Free allocated memory
            cudaFree(d_rgb_image);
            cudaFree(d_grey_image);
            stbi_image_free(rgb_image);
            free(grey_image_filename);
            free(grey_image);

        }

        return 0;

    }


Overwriting src/greyScale_gpu.cu


In [58]:
!nvcc src/greyScale_gpu.cu -o src/greyScale_gpu -lm
!./src/greyScale_gpu images/HD.jpg

src/stb_image.h(4262): warning #550-D: variable "old_limit" was set but never used

src/stb_image.h(5184): warning #550-D: variable "idata_limit_old" was set but never used

src/stb_image.h(6971): warning #550-D: variable "out_size" was set but never used

src/stb_image.h(6972): warning #550-D: variable "delays_size" was set but never used

[info] Processing images/HD.jpg
[info] images/HD: width=1280, height=720, nchannels=3
Tiempo: 0.040417


In [47]:
%%writefile createTable.sh
    #!/bin/sh

    algorithms=("greyScale_cpu" "greyScale_gpu")
    resolutions=("SD" "HD" "FHD" "4k" "8k")

    # Print the table headers
    printf "%-21s" "" > table.txt
    for res in "${resolutions[@]}"; do
    printf "%-10s" "$res" >> table.txt
    done
    printf "\n" >> table.txt

    for alg in "${algorithms[@]}"; do

        # Print row header
        printf "%-21s" "$alg" >> table.txt

        # Execute the algorithm for each resolution
        for res in "${resolutions[@]}"; do
            img="images/$res.jpg"
            echo "Processing $img file with $alg algorithm..."
            # Extract the processing time from the programs's output
            processing_time=$(./src/"$alg" $img | tail -n 1 | cut -d' ' -f2)
            echo "$img_name processing time: $processing_time"
            # Calculate the number of FPS, this is, the inverse of the processing time
            printf "%-10s" "$processing_time" >> table.txt
        done

        printf "\n" >> table.txt

    done

Overwriting createTable.sh


In [51]:
!chmod +x createTable.sh
!./createTable.sh

Processing images/SD.jpg file with greyScale_cpu algorithm...
 processing time: 0.003263
Processing images/HD.jpg file with greyScale_cpu algorithm...
 processing time: 0.011346
Processing images/FHD.jpg file with greyScale_cpu algorithm...
 processing time: 0.024920
Processing images/4k.jpg file with greyScale_cpu algorithm...
 processing time: 0.091866
Processing images/8k.jpg file with greyScale_cpu algorithm...
 processing time: 0.389660
Processing images/SD.jpg file with greyScale_gpu algorithm...
 processing time: 0.011104
Processing images/HD.jpg file with greyScale_gpu algorithm...
 processing time: 0.044729
Processing images/FHD.jpg file with greyScale_gpu algorithm...
 processing time: 0.092928
Processing images/4k.jpg file with greyScale_gpu algorithm...
 processing time: 0.342511
Processing images/8k.jpg file with greyScale_gpu algorithm...
 processing time: 1.558283
